<a href="https://colab.research.google.com/github/Paparare/AAL_Dep/blob/main/AAL_DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install conllu

In [13]:
import spacy
from spacy.tokens import Doc
from spacy.vocab import Vocab
from spacy.training import Corpus
from spacy.util import compile_infix_regex
from spacy.tokens import DocBin
from spacy.training.example import Example
import random
import conllu
from spacy.util import minibatch, compounding
import random

## **Train on 500 (aal250+wh250) dataset**

In [36]:
# Load the dataset
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    sentences = conllu.parse(text)
    return sentences

def conllu_to_docbin(input_path, output_path):
    nlp = spacy.blank('en')
    print("Created blank 'en' model")
    db = DocBin()

    for parsed_sentence in load_data(input_path):
        words = [token['form'] for token in parsed_sentence]
        heads = [token['head'] for token in parsed_sentence]
        deps = [token['deprel'] for token in parsed_sentence]

        doc = Doc(nlp.vocab, words=words)

        for i, token in enumerate(parsed_sentence):
            if heads[i] is not None and heads[i] > 0:
                doc[i].head = doc[heads[i] - 1]
            doc[i].dep_ = deps[i]

        db.add(doc)

    db.to_disk(output_path)
    print(f"Saved DocBin to {output_path}")

input_file = "all500_gold.conllu"
output_file = "output/all500_gold.spacy"

conllu_to_docbin(input_file, output_file)

print("Conversion complete")


Created blank 'en' model
Saved DocBin to output/all500_gold.spacy
Conversion complete


In [37]:

nlp = spacy.load("en_core_web_sm")

with nlp.disable_pipes(*[pipe for pipe in nlp.pipe_names if pipe not in ["parser", "tok2vec"]]):
    optimizer = nlp.resume_training()

    train_corpus = Corpus("output/all500_gold.spacy")
    train_examples = list(train_corpus(nlp))

    for iteration in range(30):
        random.shuffle(train_examples)
        losses = {}
        batches = minibatch(train_examples, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            nlp.update(batch, drop=0.5, losses=losses, sgd=optimizer)

        print(f"Iteration {iteration}, Losses: {losses}")

Iteration 0, Losses: {'tok2vec': 38428.695507347584, 'parser': 6646.2635613981765}
Iteration 1, Losses: {'tok2vec': 20638.07825422287, 'parser': 5166.6044109085915}
Iteration 2, Losses: {'tok2vec': 16445.945892333984, 'parser': 4713.473495646052}
Iteration 3, Losses: {'tok2vec': 13989.513061970472, 'parser': 4390.95919831109}
Iteration 4, Losses: {'tok2vec': 11931.193512827158, 'parser': 4035.1345274132827}
Iteration 5, Losses: {'tok2vec': 10985.20815642178, 'parser': 3829.8023581118846}
Iteration 6, Losses: {'tok2vec': 10754.943565130234, 'parser': 3671.2738578315607}
Iteration 7, Losses: {'tok2vec': 9567.782648071647, 'parser': 3417.8675362602203}
Iteration 8, Losses: {'tok2vec': 8859.520077776164, 'parser': 3312.0927643062837}
Iteration 9, Losses: {'tok2vec': 9053.932231388986, 'parser': 3197.3873333335437}
Iteration 10, Losses: {'tok2vec': 8532.377173325047, 'parser': 3027.662936807563}
Iteration 11, Losses: {'tok2vec': 8403.350032903254, 'parser': 2958.7527236714727}
Iteration 12,

In [38]:
nlp.to_disk("output/all_tokenizer")

print("Training completed")

Training completed


In [39]:
# Load the dataset
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    sentences = conllu.parse(text)
    return sentences

def conllu_to_docbin(input_path, output_path):
    nlp = spacy.blank('en')
    print("Created blank 'en' model")
    db = DocBin()

    for parsed_sentence in load_data(input_path):
        words = [token['form'] for token in parsed_sentence]
        heads = [token['head'] for token in parsed_sentence]
        deps = [token['deprel'] for token in parsed_sentence]
        doc = Doc(nlp.vocab, words=words)

        for i, token in enumerate(parsed_sentence):
            if heads[i] is not None and heads[i] > 0:
                doc[i].head = doc[heads[i] - 1]
            doc[i].dep_ = deps[i]

        db.add(doc)

    db.to_disk(output_path)
    print(f"Saved DocBin to {output_path}")

input_file = "aa250_gold.conllu"
output_file = "output/aa250_gold.spacy"

conllu_to_docbin(input_file, output_file)

print("Conversion complete")

Created blank 'en' model
Saved DocBin to output/aa250_gold.spacy
Conversion complete


In [46]:
from spacy.scorer import Scorer

nlp = spacy.load("/content/output/all_tokenizer")

eval_corpus = Corpus("/content/output/aa250_gold.spacy")
eval_examples = list(eval_corpus(nlp))

# Evaluate the model
for example in eval_examples:
    predicted = nlp(example.predicted.text)
    example.predicted = predicted

scorer = Scorer(nlp)
scores = scorer.score(eval_examples)

print("Scores:", scores)

Scores: {'token_acc': None, 'token_p': None, 'token_r': None, 'token_f': None, 'tag_acc': None, 'sents_p': 0.9280155642023347, 'sents_r': 0.8688524590163934, 'sents_f': 0.8974600188146754, 'dep_uas': 0.8517110266159696, 'dep_las': 0.49102187397975844, 'dep_las_per_type': {'discourse': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nsubj': {'p': 0.886685552407932, 'r': 0.9178885630498533, 'f': 0.9020172910662825}, 'root': {'p': 0.4083044982698962, 'r': 0.944, 'f': 0.5700483091787439}, 'obj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, '_': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'parataxis': {'p': 0.7727272727272727, 'r': 0.1878453038674033, 'f': 0.3022222222222222}, 'det': {'p': 0.9502762430939227, 'r': 0.9828571428571429, 'f': 0.9662921348314607}, 'case': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'obl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advmod': {'p': 0.75, 'r': 0.9020979020979021, 'f': 0.819047619047619}, 'compound': {'p': 0.8888888888888888, 'r': 0.7272727272727273, 'f': 0.7999999999999999}, 'dep': {'p': 0.0, 'r': 0.0, 'f': 0.

Scores: {'token_acc': None, 'token_p': None, 'token_r': None, 'token_f': None, 'tag_acc': None, 'sents_p': 0.9280155642023347, 'sents_r': 0.8688524590163934, 'sents_f': 0.8974600188146754, 'dep_uas': 0.8517110266159696, 'dep_las': 0.49102187397975844, 'dep_las_per_type': {'discourse': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nsubj': {'p': 0.886685552407932, 'r': 0.9178885630498533, 'f': 0.9020172910662825}, 'root': {'p': 0.4083044982698962, 'r': 0.944, 'f': 0.5700483091787439}, 'obj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, '_': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'parataxis': {'p': 0.7727272727272727, 'r': 0.1878453038674033, 'f': 0.3022222222222222}, 'det': {'p': 0.9502762430939227, 'r': 0.9828571428571429, 'f': 0.9662921348314607}, 'case': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'obl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advmod': {'p': 0.75, 'r': 0.9020979020979021, 'f': 0.819047619047619}, 'compound': {'p': 0.8888888888888888, 'r': 0.7272727272727273, 'f': 0.7999999999999999}, 'dep': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nmod_poss': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'appos': {'p': 1.0, 'r': 0.75, 'f': 0.8571428571428571}, 'compound_prt': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'vocative': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'mark': {'p': 0.8050847457627118, 'r': 0.9313725490196079, 'f': 0.8636363636363636}, 'advcl': {'p': 0.7142857142857143, 'r': 0.6481481481481481, 'f': 0.6796116504854369}, 'cop': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'cc': {'p': 0.9344262295081968, 'r': 0.8769230769230769, 'f': 0.9047619047619049}, 'conj': {'p': 0.9166666666666666, 'r': 0.6875, 'f': 0.7857142857142857}, 'acl': {'p': 0.9, 'r': 1.0, 'f': 0.9473684210526316}, 'aux': {'p': 0.75, 'r': 1.0, 'f': 0.8571428571428571}, 'acl_relcl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'xcomp': {'p': 0.8854961832061069, 'r': 0.928, 'f': 0.9062500000000001}, 'amod': {'p': 0.978494623655914, 'r': 0.9578947368421052, 'f': 0.9680851063829788}, 'nmod': {'p': 0.7941176470588235, 'r': 0.7941176470588235, 'f': 0.7941176470588235}, 'ccomp': {'p': 0.8717948717948718, 'r': 0.8095238095238095, 'f': 0.8395061728395062}, 'iobj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'expl': {'p': 0.6666666666666666, 'r': 0.6666666666666666, 'f': 0.6666666666666666}, 'nummod': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'obl_tmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'obl_npmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'fixed': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'det_predet': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'list': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nmod_tmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'flat': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'csubj': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'aux_pass': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'pos_acc': None, 'morph_acc': None, 'morph_micro_p': None, 'morph_micro_r': None, 'morph_micro_f': None, 'morph_per_feat': None, 'lemma_acc': None, 'ents_p': None, 'ents_r': None, 'ents_f': None, 'ents_per_type': None}


## **Train on WH250 dataset**

In [47]:
# Load the dataset
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    sentences = conllu.parse(text)
    return sentences

def conllu_to_docbin(input_path, output_path):
    nlp = spacy.blank('en')
    print("Created blank 'en' model")
    db = DocBin()

    for parsed_sentence in load_data(input_path):
        words = [token['form'] for token in parsed_sentence]
        heads = [token['head'] for token in parsed_sentence]
        deps = [token['deprel'] for token in parsed_sentence]

        doc = Doc(nlp.vocab, words=words)

        for i, token in enumerate(parsed_sentence):
            if heads[i] is not None and heads[i] > 0:
                doc[i].head = doc[heads[i] - 1]
            doc[i].dep_ = deps[i]

        db.add(doc)

    db.to_disk(output_path)
    print(f"Saved DocBin to {output_path}")

input_file = "wh250_gold.conllu"
output_file = "output/wh250_gold.spacy"

conllu_to_docbin(input_file, output_file)

print("Conversion complete")

Created blank 'en' model
Saved DocBin to output/wh250_gold.spacy
Conversion complete


In [48]:
nlp = spacy.load("en_core_web_sm")

with nlp.disable_pipes(*[pipe for pipe in nlp.pipe_names if pipe not in ["parser", "tok2vec"]]):
    optimizer = nlp.resume_training()

    train_corpus = Corpus("output/wh250_gold.spacy")
    train_examples = list(train_corpus(nlp))

    for iteration in range(30):
        random.shuffle(train_examples)
        losses = {}
        batches = minibatch(train_examples, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            nlp.update(batch, drop=0.5, losses=losses, sgd=optimizer)

        print(f"Iteration {iteration}, Losses: {losses}")

Iteration 0, Losses: {'tok2vec': 24330.326726436615, 'parser': 3596.378587731033}
Iteration 1, Losses: {'tok2vec': 14136.28227853775, 'parser': 2848.379680356041}
Iteration 2, Losses: {'tok2vec': 10368.99441409111, 'parser': 2502.7941889755416}
Iteration 3, Losses: {'tok2vec': 9374.442899689078, 'parser': 2296.8618423559797}
Iteration 4, Losses: {'tok2vec': 8507.319868206978, 'parser': 2140.422116699302}
Iteration 5, Losses: {'tok2vec': 7395.98690778017, 'parser': 2019.594276458954}
Iteration 6, Losses: {'tok2vec': 7250.293875068426, 'parser': 1977.4128431193599}
Iteration 7, Losses: {'tok2vec': 6434.768035650253, 'parser': 1823.6382023935307}
Iteration 8, Losses: {'tok2vec': 5756.613754563034, 'parser': 1730.838154066566}
Iteration 9, Losses: {'tok2vec': 5889.736658871174, 'parser': 1730.5250339537763}
Iteration 10, Losses: {'tok2vec': 5402.47863124311, 'parser': 1646.6211541451876}
Iteration 11, Losses: {'tok2vec': 5354.026055032387, 'parser': 1589.5645247592147}
Iteration 12, Losses

In [49]:
nlp.to_disk("output/wh_tokenizer")

print("Training completed")

Training completed


In [50]:
from spacy.scorer import Scorer

nlp = spacy.load("/content/output/wh_tokenizer")

eval_corpus = Corpus("/content/output/aa250_gold.spacy")
eval_examples = list(eval_corpus(nlp))

# Evaluate the model
for example in eval_examples:
    predicted = nlp(example.predicted.text)
    example.predicted = predicted

scorer = Scorer(nlp)
scores = scorer.score(eval_examples)

print("Scores:", scores)

Scores: {'token_acc': None, 'token_p': None, 'token_r': None, 'token_f': None, 'tag_acc': None, 'sents_p': 0.8874045801526718, 'sents_r': 0.8469945355191257, 'sents_f': 0.8667287977632806, 'dep_uas': 0.6482003649029691, 'dep_las': 0.32793853196011113, 'dep_las_per_type': {'discourse': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nsubj': {'p': 0.688953488372093, 'r': 0.6950146627565983, 'f': 0.6919708029197081}, 'root': {'p': 0.3013698630136986, 'r': 0.704, 'f': 0.422062350119904}, 'obj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, '_': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'parataxis': {'p': 0.20930232558139536, 'r': 0.049723756906077346, 'f': 0.08035714285714285}, 'det': {'p': 0.7785234899328859, 'r': 0.6628571428571428, 'f': 0.7160493827160492}, 'case': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'obl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advmod': {'p': 0.42857142857142855, 'r': 0.5454545454545454, 'f': 0.4799999999999999}, 'compound': {'p': 0.12751677852348994, 'r': 0.5757575757575758, 'f': 0.2087912087912088}, 'dep': {'p': 0.

Scores: {'token_acc': None, 'token_p': None, 'token_r': None, 'token_f': None, 'tag_acc': None, 'sents_p': 0.8874045801526718, 'sents_r': 0.8469945355191257, 'sents_f': 0.8667287977632806, 'dep_uas': 0.6482003649029691, 'dep_las': 0.32793853196011113, 'dep_las_per_type': {'discourse': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nsubj': {'p': 0.688953488372093, 'r': 0.6950146627565983, 'f': 0.6919708029197081}, 'root': {'p': 0.3013698630136986, 'r': 0.704, 'f': 0.422062350119904}, 'obj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, '_': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'parataxis': {'p': 0.20930232558139536, 'r': 0.049723756906077346, 'f': 0.08035714285714285}, 'det': {'p': 0.7785234899328859, 'r': 0.6628571428571428, 'f': 0.7160493827160492}, 'case': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'obl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advmod': {'p': 0.42857142857142855, 'r': 0.5454545454545454, 'f': 0.4799999999999999}, 'compound': {'p': 0.12751677852348994, 'r': 0.5757575757575758, 'f': 0.2087912087912088}, 'dep': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'xcomp': {'p': 0.5630252100840336, 'r': 0.536, 'f': 0.5491803278688525}, 'nmod': {'p': 0.2978723404255319, 'r': 0.4117647058823529, 'f': 0.345679012345679}, 'nmod_poss': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'appos': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'compound_prt': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'vocative': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'mark': {'p': 0.6632653061224489, 'r': 0.6372549019607843, 'f': 0.6499999999999999}, 'advcl': {'p': 0.34782608695652173, 'r': 0.14814814814814814, 'f': 0.20779220779220778}, 'acl': {'p': 0.16666666666666666, 'r': 0.1111111111111111, 'f': 0.13333333333333333}, 'cop': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'cc': {'p': 0.7619047619047619, 'r': 0.7384615384615385, 'f': 0.75}, 'conj': {'p': 0.45, 'r': 0.421875, 'f': 0.435483870967742}, 'aux': {'p': 0.625, 'r': 0.7246376811594203, 'f': 0.6711409395973154}, 'acl_relcl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'amod': {'p': 0.5701754385964912, 'r': 0.6842105263157895, 'f': 0.6220095693779903}, 'ccomp': {'p': 0.36585365853658536, 'r': 0.35714285714285715, 'f': 0.3614457831325302}, 'nummod': {'p': 0.5384615384615384, 'r': 0.875, 'f': 0.6666666666666667}, 'iobj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'expl': {'p': 0.125, 'r': 0.16666666666666666, 'f': 0.14285714285714288}, 'obl_tmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'obl_npmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'fixed': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'det_predet': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'list': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nmod_tmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'flat': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'csubj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'aux_pass': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'pos_acc': None, 'morph_acc': None, 'morph_micro_p': None, 'morph_micro_r': None, 'morph_micro_f': None, 'morph_per_feat': None, 'lemma_acc': None, 'ents_p': None, 'ents_r': None, 'ents_f': None, 'ents_per_type': None}


## **Train on aal250 and test on the same dataset**

In [51]:
nlp = spacy.load("/content/output/tokenizer")

eval_corpus = Corpus("/content/output/aa250_gold.spacy")
eval_examples = list(eval_corpus(nlp))

# Evaluate the model
for example in eval_examples:
    predicted = nlp(example.predicted.text)
    example.predicted = predicted

scorer = Scorer(nlp)
scores = scorer.score(eval_examples)

print("Scores:", scores)

Scores: {'token_acc': None, 'token_p': None, 'token_r': None, 'token_f': None, 'tag_acc': None, 'sents_p': 0.9137931034482759, 'sents_r': 0.8688524590163934, 'sents_f': 0.8907563025210085, 'dep_uas': 0.8602649006622517, 'dep_las': 0.500571055637135, 'dep_las_per_type': {'discourse': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nsubj': {'p': 0.8931506849315068, 'r': 0.9560117302052786, 'f': 0.923512747875354}, 'root': {'p': 0.4071550255536627, 'r': 0.956, 'f': 0.5710872162485066}, 'obj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, '_': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'parataxis': {'p': 0.7857142857142857, 'r': 0.18232044198895028, 'f': 0.29596412556053814}, 'det': {'p': 0.9613259668508287, 'r': 0.9942857142857143, 'f': 0.9775280898876405}, 'case': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'obl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advmod': {'p': 0.7362637362637363, 'r': 0.9370629370629371, 'f': 0.8246153846153847}, 'compound': {'p': 0.8928571428571429, 'r': 0.7575757575757576, 'f': 0.819672131147541}, 'dep': {'p': 0.0, 'r'

Scores: {'token_acc': None, 'token_p': None, 'token_r': None, 'token_f': None, 'tag_acc': None, 'sents_p': 0.9137931034482759, 'sents_r': 0.8688524590163934, 'sents_f': 0.8907563025210085, 'dep_uas': 0.8602649006622517, 'dep_las': 0.500571055637135, 'dep_las_per_type': {'discourse': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nsubj': {'p': 0.8931506849315068, 'r': 0.9560117302052786, 'f': 0.923512747875354}, 'root': {'p': 0.4071550255536627, 'r': 0.956, 'f': 0.5710872162485066}, 'obj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, '_': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'parataxis': {'p': 0.7857142857142857, 'r': 0.18232044198895028, 'f': 0.29596412556053814}, 'det': {'p': 0.9613259668508287, 'r': 0.9942857142857143, 'f': 0.9775280898876405}, 'case': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'obl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advmod': {'p': 0.7362637362637363, 'r': 0.9370629370629371, 'f': 0.8246153846153847}, 'compound': {'p': 0.8928571428571429, 'r': 0.7575757575757576, 'f': 0.819672131147541}, 'dep': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nmod_poss': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'appos': {'p': 1.0, 'r': 0.375, 'f': 0.5454545454545454}, 'compound_prt': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'vocative': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'mark': {'p': 0.9166666666666666, 'r': 0.9705882352941176, 'f': 0.9428571428571428}, 'advcl': {'p': 0.78, 'r': 0.7222222222222222, 'f': 0.7500000000000001}, 'cop': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'cc': {'p': 0.9375, 'r': 0.9230769230769231, 'f': 0.9302325581395349}, 'conj': {'p': 0.9230769230769231, 'r': 0.75, 'f': 0.8275862068965517}, 'acl': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'aux': {'p': 0.711340206185567, 'r': 1.0, 'f': 0.8313253012048193}, 'acl_relcl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'xcomp': {'p': 0.9590163934426229, 'r': 0.936, 'f': 0.9473684210526315}, 'amod': {'p': 0.9787234042553191, 'r': 0.968421052631579, 'f': 0.9735449735449735}, 'nmod': {'p': 1.0, 'r': 0.5882352941176471, 'f': 0.7407407407407407}, 'pcomp': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'ccomp': {'p': 0.8947368421052632, 'r': 0.8095238095238095, 'f': 0.8500000000000001}, 'iobj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'expl': {'p': 1.0, 'r': 0.8333333333333334, 'f': 0.9090909090909091}, 'nummod': {'p': 1.0, 'r': 0.875, 'f': 0.9333333333333333}, 'obl_tmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'obl_npmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'fixed': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'det_predet': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'list': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nmod_tmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'flat': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'csubj': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'aux_pass': {'p': 0.0, 'r': 0.0, 'f': 0.0}}, 'pos_acc': None, 'morph_acc': None, 'morph_micro_p': None, 'morph_micro_r': None, 'morph_micro_f': None, 'morph_per_feat': None, 'lemma_acc': None, 'ents_p': None, 'ents_r': None, 'ents_f': None, 'ents_per_type': None}
